<h1>Análise de Base de Dados sobre o Fluxo de rede que passa por um IP que sofre um ataque de força bruta</h1>

In [1]:
%matplotlib inline
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

In [10]:
#Leitura do Arquivo com dados da rede com ocorrencia do ataque
baseForcaBrutaCA = pd.read_csv("C:/Users/camil/Downloads/camilaServidorComAt.csv")

#imprimindo dataframe
#baseForcaBrutaCA

listaIpDestinoSemOrdenacao = baseForcaBrutaCA['destination.nat.ip'].values
listaIpDestinoOrdenado = sorted(listaIpDestinoSemOrdenacao)

cont = 0

listaIpsDestino = [] #lista com ip's que tentaram conexão
rep = 0
#iteração para salvar nas listas os ip's com seus respectivos números de tentativas
for k in range(0,len(listaIpDestinoOrdenado)-1):
    if(listaIpDestinoOrdenado[k] == listaIpDestinoOrdenado[k+1]):
        rep += 1
        if (k == len(listaIpDestinoOrdenado)-2):
            listaIpsDestino.insert(k, listaIpDestinoOrdenado[k])          
    else:
        listaIpsDestino.insert(k, listaIpDestinoOrdenado[k])
        rep = 0
        
for cont in range(len(listaIpsDestino)):
    #Mascara e filtro para obter dados do dataframe relacionados somente a determinado ip quando está sofrendo um suposto ataque de força bruta, por fim salvando em um novo dataframe
    baseForcaBrutaCA_mask=baseForcaBrutaCA['destination.nat.ip']==listaIpsDestino[cont]
    filtered_baseForcaBrutaCA = baseForcaBrutaCA[baseForcaBrutaCA_mask]
    baseForcaBrutaCA_FluxoAtacanteVitima = filtered_baseForcaBrutaCA [['source.nat.ip' ,'destination.nat.ip', 'flow.tcp_flags']]
    baseForcaBrutaCA_FluxoAtacanteVitima

    #Salvando em um vetor as flags que chegaram ao ip destino
    vetorFlagsAtacanteToVitima = baseForcaBrutaCA_FluxoAtacanteVitima['flow.tcp_flags'].values

    #Salvando em um vetor os ip's que tentaram uma conexão com o ip destino 
    vetorCA = baseForcaBrutaCA_FluxoAtacanteVitima['source.nat.ip'].values

    #Ordenando todos os ip's que tentaram conexão com o ip destino
    vetorCAOrdenado = sorted(vetorCA)

    listaIPAtacanteCA = [] #lista com ip's que tentaram conexão
    listaNumeroOcorrenciasCA = [] #lista com o número de tentativas de conexão de cada ip origem com o ip destino
    rep = 0
    #iteração para salvar nas listas os ip's com seus respectivos números de tentativas
    for k in range(0,len(vetorCAOrdenado)-1):
        if(vetorCAOrdenado[k] == vetorCAOrdenado[k+1]):
            rep += 1
            if (k == len(vetorCAOrdenado)-2):
                listaIPAtacanteCA.insert(k, vetorCAOrdenado[k])
                listaNumeroOcorrenciasCA.insert(k, rep+1)          
        else:
            listaIPAtacanteCA.insert(k, vetorCAOrdenado[k])
            listaNumeroOcorrenciasCA.insert(k, rep+1)
            rep = 0
        
    listaIpsFluxoMaiorDez = [] #Lista dos ip's origem que tentaram por mais de 10 vezes uma conexão com o ip destino
    i = 0
    #iteração para salvar na lista os ip's que tentaram por mais de 10 vezes uma conexão com o ip destino
    for i in range(len(listaNumeroOcorrenciasCA)):
        if listaNumeroOcorrenciasCA[i] > 10:
            listaIpsFluxoMaiorDez.insert (i, listaIPAtacanteCA[i])
        
    i = 0
    ip = 0
    aux = 0
    #iteração para verificar a sequencia de flags enviadas pelo ip origem, sendo que
    #se o mesmo enviou o flag "SYN" sequencialmente, sem ter qualquer outro flag, podemos concluir q o mesmo está tentando uma conexão sem sucesso, o que pode identificar um ataque
    for ip in range(len(listaIpsFluxoMaiorDez)):
        for i in range(len(listaNumeroOcorrenciasCA)):
            if vetorCA[i] == listaIpsFluxoMaiorDez[ip]:
                if vetorFlagsAtacanteToVitima[i] == '["SYN"]':
                    aux = aux + 1 
                    if aux > 10:
                        print ("Possível ataque ocorrendo do Ip Origem ", vetorCA[i], " para o Ip Destino ", listaIpsDestino[cont])
                        break
                else:
                    aux = 0

Possível ataque ocorrendo do Ip Origem  177.124.72.16  para o Ip Destino  177.124.77.45
